**GENDER CLASSIFICATION USING LSTM**

IMPORTING REQUIRED LIBRARIES

In [10]:

from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
import pandas as pd
import numpy as np
import os

In [27]:
#parameters
maxlen = 30
labels = 2

Google drive Mount for Importing input data CSV

In [11]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Path from Google drive

In [12]:
path = "/content/gdrive/MyDrive/Colab Notebooks/name_gender.csv"


In [13]:
input = pd.read_csv(path)

In [14]:
df1=input
df1

,name,gender
0,Aaban&&,M
1,Aabha*,F
2,Aabid,M
3,Aabriella,F
4,Aada_,F
...,...,...
95020,Zyvion,M
95021,Zyvon,M
95022,Zyyanna,F
95023,Zyyon,M


Filtering only alphabets aand removing numerics and spacial Characters

In [15]:
def alphabets(element):
    
    return "".join(filter(str.isalpha, element))

In [16]:
df1.name = [alphabets(x) for x in df1.name]
df1['name']=df1['name'].str.lower()
df1

,name,gender
0,aaban,M
1,aabha,F
2,aabid,M
3,aabriella,F
4,aada,F
...,...,...
95020,zyvion,M
95021,zyvon,M
95022,zyyanna,F
95023,zyyon,M


In [17]:
input=df1

In [18]:
input.columns = ['name','m_or_f']
input['namelen']= [len(str(i)) for i in input['name']]
input1 = input[(input['namelen'] >= 2) ]


In [19]:
input1.groupby('m_or_f')['name'].count()

m_or_f
F    60304
M    34721
Name: name, dtype: int64

In [20]:
names = input['name']
gender = input['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
len_vocab = len(vocab)

Identifying Vocabulary and length of vocabulary words

In [21]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of input is ",len(input1))

{'w', 'u', 'd', 'v', 'a', 't', 'n', 'p', 'r', 'g', 'q', 'y', 'x', 'b', 's', 'k', 'END', ' ', 'l', 'h', 'c', 'f', 'e', 'z', 'i', 'm', 'j', 'o'}
vocab length is  28
length of input is  95025


In [22]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [23]:
print(char_index)

{'w': 0, 'u': 1, 'd': 2, 'v': 3, 'a': 4, 't': 5, 'n': 6, 'p': 7, 'r': 8, 'g': 9, 'q': 10, 'y': 11, 'x': 12, 'b': 13, 's': 14, 'k': 15, 'END': 16, ' ': 17, 'l': 18, 'h': 19, 'c': 20, 'f': 21, 'e': 22, 'z': 23, 'i': 24, 'm': 25, 'j': 26, 'o': 27}


In [24]:
#train test split
msk = np.random.rand(len(input1)) < 0.8
train = input1[msk]
test = input1[~msk]    

In [28]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
train_X = []
trunc_train_name = [str(i)[0:30] for i in train.name]
for i in trunc_train_name:
    tmp = [char_index[j] for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(char_index["END"])
    train_X.append(tmp)

Changing datatype to numpy array

In [29]:
train_X=np.asarray(train_X)

In [30]:
train_X.shape

(76071, 30)

In [31]:
def set_flag(i):
    tmp = np.zeros(28);
    tmp[i] = 1
    return(tmp)

In [32]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [33]:
#take input upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.m_or_f:
    if i == 'M':
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    

In [34]:
train_X=np.asarray(train_X)

In [35]:
train_X.shape

(76071, 30, 28)

In [36]:
train_Y=np.asarray(train_Y)

In [37]:
train_Y.shape

(76071, 2)

**Model building**

In [38]:
#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen,len_vocab)))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

Build model...


In [39]:
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test.m_or_f:
    if i == 'M':
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])

In [40]:
test_X=np.asarray(test_X)

In [41]:
test_Y=np.asarray(test_Y)

In [42]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(18954, 30, 28)
(18954, 2)


**Model Fitting and Hyperparameter Tuning**

In [43]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,epochs=10,validation_data=(test_X, test_Y))

Epoch 1/10
77/77 [==============================] - 36s 367ms/step - loss: 0.6018 - accuracy: 0.6751 - val_loss: 0.5789 - val_accuracy: 0.6964
Epoch 2/10
77/77 [==============================] - 27s 357ms/step - loss: 0.4944 - accuracy: 0.7638 - val_loss: 0.4632 - val_accuracy: 0.7896
Epoch 3/10
77/77 [==============================] - 28s 360ms/step - loss: 0.4346 - accuracy: 0.8019 - val_loss: 0.3919 - val_accuracy: 0.8262
Epoch 4/10
77/77 [==============================] - 28s 360ms/step - loss: 0.3840 - accuracy: 0.8310 - val_loss: 0.3745 - val_accuracy: 0.8317
Epoch 5/10
77/77 [==============================] - 28s 359ms/step - loss: 0.3642 - accuracy: 0.8409 - val_loss: 0.3507 - val_accuracy: 0.8472
Epoch 6/10
77/77 [==============================] - 28s 360ms/step - loss: 0.3488 - accuracy: 0.8496 - val_loss: 0.3663 - val_accuracy: 0.8365
Epoch 7/10
77/77 [==============================] - 28s 359ms/step - loss: 0.3421 - accuracy: 0.8514 - val_loss: 0.3278 - val_accuracy: 0.8583

In [44]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

593/593 [==============================] - 9s 15ms/step - loss: 0.3135 - accuracy: 0.8668
Test score: 0.31346431374549866
Test accuracy: 0.8667827248573303


In [45]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.01947986, 0.9805202 ],
       [0.27311394, 0.72688603],
       [0.8353916 , 0.16460846],
       [0.16387771, 0.83612233],
       [0.53243554, 0.46756446],
       [0.65861756, 0.34138238],
       [0.10805432, 0.89194566],
       [0.04642663, 0.9535734 ],
       [0.45527634, 0.54472363]], dtype=float32)

In [46]:
name=["aadhira","aabid"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.0137256 , 0.9862744 ],
       [0.74699855, 0.25300145]], dtype=float32)

In [47]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 512)           1107968   
                                                                 
 dropout (Dropout)           (None, 30, 512)           0         
                                                                 
 lstm_1 (LSTM)               (None, 512)               2099200   
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
 activation (Activation)     (None, 2)                 0         
                                                                 
Total params: 3,208,194
Trainable params: 3,208,194
Non-

**Hyperparameter Tuning**

In [50]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,epochs=50,validation_data=(test_X, test_Y))

Epoch 1/50
77/77 [==============================] - 28s 361ms/step - loss: 0.2954 - accuracy: 0.8752 - val_loss: 0.2967 - val_accuracy: 0.8734
Epoch 2/50
77/77 [==============================] - 28s 366ms/step - loss: 0.2875 - accuracy: 0.8779 - val_loss: 0.2956 - val_accuracy: 0.8732
Epoch 3/50
77/77 [==============================] - 28s 360ms/step - loss: 0.2769 - accuracy: 0.8837 - val_loss: 0.2974 - val_accuracy: 0.8739
Epoch 4/50
77/77 [==============================] - 28s 360ms/step - loss: 0.2754 - accuracy: 0.8845 - val_loss: 0.2866 - val_accuracy: 0.8807
Epoch 5/50
77/77 [==============================] - 28s 359ms/step - loss: 0.2638 - accuracy: 0.8893 - val_loss: 0.2747 - val_accuracy: 0.8855
Epoch 6/50
77/77 [==============================] - 28s 359ms/step - loss: 0.2564 - accuracy: 0.8933 - val_loss: 0.2809 - val_accuracy: 0.8827
Epoch 7/50
77/77 [==============================] - 28s 360ms/step - loss: 0.2509 - accuracy: 0.8960 - val_loss: 0.2743 - val_accuracy: 0.8900

**Hyperparameter tuning**

In [51]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,epochs=10,validation_data=(test_X, test_Y))

Epoch 1/10
77/77 [==============================] - 28s 361ms/step - loss: 0.0309 - accuracy: 0.9884 - val_loss: 0.4545 - val_accuracy: 0.9060
Epoch 2/10
77/77 [==============================] - 28s 358ms/step - loss: 0.0315 - accuracy: 0.9886 - val_loss: 0.4448 - val_accuracy: 0.8981
Epoch 3/10
77/77 [==============================] - 28s 360ms/step - loss: 0.0471 - accuracy: 0.9822 - val_loss: 0.4480 - val_accuracy: 0.9049
Epoch 4/10
77/77 [==============================] - 28s 359ms/step - loss: 0.0312 - accuracy: 0.9881 - val_loss: 0.4819 - val_accuracy: 0.9036
Epoch 5/10
77/77 [==============================] - 28s 359ms/step - loss: 0.0281 - accuracy: 0.9899 - val_loss: 0.4682 - val_accuracy: 0.9063
Epoch 6/10
77/77 [==============================] - 28s 360ms/step - loss: 0.0207 - accuracy: 0.9924 - val_loss: 0.4880 - val_accuracy: 0.9081
Epoch 7/10
77/77 [==============================] - 28s 359ms/step - loss: 0.0254 - accuracy: 0.9909 - val_loss: 0.4428 - val_accuracy: 0.9081

In [52]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,epochs=20,validation_data=(test_X, test_Y))

Epoch 1/20
77/77 [==============================] - 28s 361ms/step - loss: 0.0172 - accuracy: 0.9939 - val_loss: 0.5043 - val_accuracy: 0.9053
Epoch 2/20
77/77 [==============================] - 28s 361ms/step - loss: 0.0177 - accuracy: 0.9941 - val_loss: 0.4551 - val_accuracy: 0.9053
Epoch 3/20
77/77 [==============================] - 28s 360ms/step - loss: 0.0208 - accuracy: 0.9927 - val_loss: 0.4560 - val_accuracy: 0.9079
Epoch 4/20
77/77 [==============================] - 28s 359ms/step - loss: 0.0191 - accuracy: 0.9937 - val_loss: 0.5285 - val_accuracy: 0.9069
Epoch 5/20
77/77 [==============================] - 28s 359ms/step - loss: 0.0171 - accuracy: 0.9944 - val_loss: 0.5029 - val_accuracy: 0.9051
Epoch 6/20
77/77 [==============================] - 28s 360ms/step - loss: 0.0226 - accuracy: 0.9922 - val_loss: 0.5019 - val_accuracy: 0.9074
Epoch 7/20
77/77 [==============================] - 28s 359ms/step - loss: 0.0181 - accuracy: 0.9939 - val_loss: 0.4415 - val_accuracy: 0.9078

In [53]:
batch_size=1000
model.fit(train_X, train_Y,batch_size=batch_size,epochs=30,validation_data=(test_X, test_Y))

Epoch 1/30
77/77 [==============================] - 28s 361ms/step - loss: 0.0086 - accuracy: 0.9973 - val_loss: 0.5246 - val_accuracy: 0.9079
Epoch 2/30
77/77 [==============================] - 28s 361ms/step - loss: 0.0153 - accuracy: 0.9952 - val_loss: 0.4876 - val_accuracy: 0.9092
Epoch 3/30
77/77 [==============================] - 28s 359ms/step - loss: 0.0095 - accuracy: 0.9969 - val_loss: 0.5502 - val_accuracy: 0.9071
Epoch 4/30
77/77 [==============================] - 28s 359ms/step - loss: 0.0092 - accuracy: 0.9968 - val_loss: 0.5164 - val_accuracy: 0.9084
Epoch 5/30
77/77 [==============================] - 28s 360ms/step - loss: 0.0099 - accuracy: 0.9968 - val_loss: 0.5074 - val_accuracy: 0.9069
Epoch 6/30
77/77 [==============================] - 28s 360ms/step - loss: 0.0192 - accuracy: 0.9933 - val_loss: 0.5072 - val_accuracy: 0.9059
Epoch 7/30
77/77 [==============================] - 28s 360ms/step - loss: 0.0090 - accuracy: 0.9971 - val_loss: 0.5249 - val_accuracy: 0.9074

In [54]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

593/593 [==============================] - 9s 15ms/step - loss: 0.5444 - accuracy: 0.9095
Test score: 0.54440838098526
Test accuracy: 0.909517765045166
